In [1]:
import os
import shutil
import argparse
import datetime
import pickle
import time

import numpy as np
import matplotlib as mpl
mpl.use('Agg')

import sys

import mxnet as mx
mx.random.seed(int(time.time()))

In [2]:
# utils.py
import socket
from functools import reduce
from scipy.special import logit
from tqdm import tqdm
import collections

In [3]:
def gpu_helper(gpu):
    if gpu >= 0 and gpu_exists(gpu):
        model_ctx = mx.gpu(gpu)
    else:
        model_ctx = mx.cpu()
    return model_ctx

def gpu_exists(gpu):
    try:
        mx.nd.zeros((1,), ctx=mx.gpu(gpu))
    except:
        return False
    return True

def calc_topic_uniqueness(top_words_idx_all_topics):
    """
    This function calculates topic uniqueness scores for a given list of topics.
    For each topics, the uniqueness is calculated as: (\sum_{i=1}^n 1/cnt(i)) / n,
    where n is the number of top words in the topic and cnt(i) is the counter for the number of times the word
    appears in the top words of all the topics.
    """
    n_topics = len(top_words_idx_all_topics)
    
    # build word_cnt_dict that is number of times the word appears in top words
    word_cnt_dict = collections.Counter()
    for i in range(n_topics):
        word_cnt_dict.update(top_words_idx_all_topics[i])
    
    uniqueness_dict = dict()
    for i in range(n_topics):
        cnt_inv_sum = 0.0
        for ind in top_words_idx_all_topics[i]:
            cnt_inv_sum += 1.0 / word_cnt_dict[ind]
        uniqueness_dict[i] = cnt_inv_sum / len(top_words_idx_all_topics)
        
    return uniqueness_dict

def request_pmi(topic_dict=None, filename='', port=1234):
    try:
        # Create a socket object
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # get local machine name
        host = socket.gethostname()
        # host = socket.gethostbyname('localhost')
        
        # connection to hostname on the port.
        s.connect((host, port))
        
        if filename != '':
            s.sendall(pickle.dumps(filename), )
        else:
            s.send(pickle.dumps(topic_dict), )
            
        data = []
        while True:
            packet = s.recv(4096)
            # time.sleep(0.1)
            # print('looking at packet # {0}'.format(len(data)))
            # print(packet)
            # print(type(packet))
            wait = len(packet)
            if not packet:
                break
            data.append(packet)
            # print('received packet # {0}'.format(len(data)))
            # time.sleep(1.0)
        res_dict = pickle.loads(b"".join(data))
        
        s.close()
        pmi_dict = res_dict['pmi_dict']
        npmi_dict = res_dict['npmi_dict']
    except:
        # print('Failed to run NPMI calc, NPMI and PMI set to 0.0')
        pmi_dict = dict()
        npmi_dict = dict()
        for k in topic_dict:
            pmi_dict[k] = 0
            npmi_dict[k] = 0
            
    return pmi_dict, npmi_dict

def print_topic_with_scores(topic_json, **kwargs):
    topic_keys = sorted(list(topic_json.keys()))
    
    sortby = kwargs.pop('sortby', None)
    if sortby is None:
        sortby = kwargs.pop('sort_by', None)
    
    if sortby in kwargs.keys():
        topic_keys = sorted(kwargs[sortby], key=kwargs[sortby].get)[::-1]
        
    entries = []
    dict_names = sorted(list(kwargs.keys()))
    header_str = 'Avg scores: '
    for dn in dict_names:
        assert isinstance(kwargs[dn], dict)
        # mean_dict 왜 갑자기 튀어나옴?
        header_str += '{}: {:.2f}'.format(dn, mean_dict(kwargs[dn]))
    for k in topic_keys:
        score_str = []
        for dn in dict_names:
            score_str.append('{:.2f}'.format(kwargs[dn][k]))
        score_str = ', '.join(score_str)
        entries.append('T{} [{}]'.format(k, score_str) + ', '.join(topic_json[k]))
        
    msg = header_str + '\n' + '\n'.join(entries)
    print(msg)
    return msg

def print_topics(topic_json, npmi_dict, topic_uniqs, data, print_topic_names=False):
    for k, v in topic_json.items():
        prefix_msg = '[ '
        if hasattr(data, 'maps') and print_topic_names:
            prefix_msg += data.maps['dim2topic'][k]
        else:
            prefix_msg += str(k)
        if hasattr(data, 'selected_topics') and print_topic_names:
            if data.maps['dim2topic'][k] in data.selected_topics:
                prefix_msg += '*'
        prefix_msg += ' - {:.5g} - {:.5g}]:'.format(topic_uniqs[k], npmi_dict[k])
        print(prefix_msg, v)
        
# 아래는 사용안하는 함수
def reverse_dict(d):
    return {v:k for k,v in d.items()}

def to_numpy(x):
    x_npy = []
    for x in X:
        if isinstance(x, list):
            x_npy += [to_numpy(x)]
        else:
            x_npy += [x.asnumpy()]
    return x_npy

def stack_numpy(X, xnew):
    for i in range(len(X)):
        if isinstance(xnew[i], list):
            X[i] = stack_numpy(x[i], xnew[i])
        else:
            X[i] = np.vstack([X[i], xnew[i]])
    return X

def get_topic_words_decoder_weights(D, data, ctx, k=10, decoder_weights=False):
    if decoder_weights:
        params = D.collect_params()
        params = params['decoder0_dense0_weight'].data().transpose()
    else:
        y = D.y_as_topics()
        params = D(y.copyto(ctx))
    top_word_ids = mx.nd.argsort(params, axis=1, is_ascend=False)[:, :k].asnumpy()
    if hasattr(data, 'id_to_word'):
        top_word_strings = [[data.is_to_word]]
    else:
        top_word_strings = [[data.maps['dim2vocab'][int(w)] for w in topic] for topic in top_word_ids]
        
    return top_word_strings

def get_topic_words(D, data, ctx, k=10):
    y, z = D.yz_as_topics()
    if z is not None:
        params = D(y.copyto(ctx), x.copyto(ctx))
    else:
        params = D(y.copyto(ctx), None)
    top_word_ids = mx.nd.argsort(params, axis=1, is_ascend=False)[:,:k].asnumpy()
    if hasattr(data, 'id_to_word'):
        top_word_strings = [[data.id_to_word[int(w)] for w in topic] for topic in top_word_ids]
    else:
        top_word_strings = [[data.maps['dim2vocab'][int(w)] for w in topic] for topic in top_word_ids]

    return top_word_strings

In [4]:
parser = argparse.ArgumentParser(description='Training WAE in MXNet')
parser.add_argument('-dom','--domain', type=str, default='twenty_news', help='domain to run', required=False)
parser.add_argument('-data','--data_path', type=str, default='', help='file path for dataset', required=False)
parser.add_argument('-max_labels','--max_labels', type=int, default=100, help='max number of topics to specify as labels for a single training document', required=False)
parser.add_argument('-max_labeled_samples','--max_labeled_samples', type=int, default=10, help='max number of labeled samples per topic', required=False)
parser.add_argument('-label_seed','--label_seed', type=lambda x: int(x) if x != 'None' else None, default=None, help='random seed for subsampling the labeled dataset', required=False)
parser.add_argument('-mod','--model', type=str, default='dirichlet', help='model to use', required=False)
parser.add_argument('-desc','--description', type=str, default='', help='description for the experiment', required=False)
parser.add_argument('-alg','--algorithm', type=str, default='standard', help='algorithm to use for training: standard', required=False)
parser.add_argument('-bs','--batch_size', type=int, default=256, help='batch_size for training', required=False)
parser.add_argument('-opt','--optim', type=str, default='Adam', help='encoder training algorithm', required=False)
parser.add_argument('-lr','--learning_rate', type=float, default=1e-4, help='learning rate', required=False)
parser.add_argument('-l2','--weight_decay', type=float, default=0., help='weight decay', required=False)
parser.add_argument('-e_nh','--enc_n_hidden', type=int, nargs='+', default=[128], help='# of hidden units for encoder or list of hiddens for each layer', required=False)
parser.add_argument('-e_nl','--enc_n_layer', type=int, default=1, help='# of hidden layers for encoder, set to -1 if passing list of n_hiddens', required=False)
parser.add_argument('-e_nonlin','--enc_nonlinearity', type=str, default='sigmoid', help='type of nonlinearity for encoder', required=False)
parser.add_argument('-e_weights','--enc_weights', type=str, default='', help='file path for encoder weights', required=False)
parser.add_argument('-e_freeze','--enc_freeze', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to freeze the encoder weights', required=False)
parser.add_argument('-lat_nonlin','--latent_nonlinearity', type=str, default='', help='type of to use prior to decoder', required=False)
parser.add_argument('-d_nh','--dec_n_hidden', type=int, nargs='+', default=[128], help='# of hidden units for decoder or list of hiddens for each layer', required=False)
parser.add_argument('-d_nl','--dec_n_layer', type=int, default=0, help='# of hidden layers for decoder', required=False)
parser.add_argument('-d_nonlin','--dec_nonlinearity', type=str, default='', help='type of nonlinearity for decoder', required=False)
parser.add_argument('-d_weights','--dec_weights', type=str, default='', help='file path for decoder weights', required=False)
parser.add_argument('-d_freeze','--dec_freeze', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to freeze the decoder weights', required=False)
parser.add_argument('-d_word_dist','--dec_word_dist', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to init decoder weights with training set word distributions', required=False)
parser.add_argument('-dis_nh','--dis_n_hidden', type=int, nargs='+', default=[128], help='# of hidden units for encoder or list of hiddens for each layer', required=False)
parser.add_argument('-dis_nl','--dis_n_layer', type=int, default=1, help='# of hidden layers for encoder, set to -1 if passing list of n_hiddens', required=False)
parser.add_argument('-dis_nonlin','--dis_nonlinearity', type=str, default='sigmoid', help='type of nonlinearity for discriminator', required=False)
parser.add_argument('-dis_y_weights','--dis_y_weights', type=str, default='', help='file path for discriminator_y weights', required=False)
parser.add_argument('-dis_z_weights','--dis_z_weights', type=str, default='', help='file path for discriminator_z weights', required=False)
parser.add_argument('-dis_freeze','--dis_freeze', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to freeze the encoder weights', required=False)
parser.add_argument('-include_w','--include_weights', type=str, nargs='*', default=[], help='weights to train on (default is all weights) -- all others are kept fixed; Ex: E.z_encoder D.decoder', required=False)
parser.add_argument('-eps','--epsilon', type=float, default=1e-8, help='epsilon param for Adam', required=False)
parser.add_argument('-mx_it','--max_iter', type=int, default=50001, help='max # of training iterations', required=False)
parser.add_argument('-train_stats_every','--train_stats_every', type=int, default=100, help='skip train_stats_every iterations between recording training stats', required=False)
parser.add_argument('-eval_stats_every','--eval_stats_every', type=int, default=100, help='skip eval_stats_every iterations between recording evaluation stats', required=False)
parser.add_argument('-ndim_y','--ndim_y', type=int, default=256, help='dimensionality of y - topic indicator', required=False)
parser.add_argument('-ndim_x','--ndim_x', type=int, default=2, help='dimensionality of p(x) - data distribution', required=False)
parser.add_argument('-saveto','--saveto', type=str, default='', help='path prefix for saving results', required=False)
parser.add_argument('-gpu','--gpu', type=int, default=-2, help='if/which gpu to use (-1: all, -2: None)', required=False)
parser.add_argument('-hybrid','--hybridize', type=lambda x: (str(x).lower() == 'true'), default=False, help='declaritive True (hybridize) or imperative False', required=False)
parser.add_argument('-full_npmi','--full_npmi', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to compute NPMI for full trajectory', required=False)
parser.add_argument('-eot','--eval_on_test', type=lambda x: (str(x).lower() == 'true'), default=False, help='whether to evaluate on the test set (True) or validation set (False)', required=False)
parser.add_argument('-verb','--verbose', type=lambda x: (str(x).lower() == 'true'), default=True, help='whether to print progress to stdout', required=False)
parser.add_argument('-dirich_alpha','--dirich_alpha', type=float, default=1e-1, help='param for Dirichlet prior', required=False)
parser.add_argument('-adverse','--adverse', type=lambda x: (str(x).lower() == 'true'), default=True, help='whether to turn on adverserial training (MMD or GAN). set to False if only train auto-encoder', required=False)
parser.add_argument('-update_enc','--update_enc', type=lambda x: (str(x).lower() == 'true'), default=True, help='whether to update encoder for unlabed_train_op()', required=False)
parser.add_argument('-labeled_loss_lambda','--labeled_loss_lambda', type=float, default=1.0, help='param for Dirichlet noise for label', required=False)
parser.add_argument('-train_mode','--train_mode', type=str, default='mmd', help="set to mmd or adv (for GAN)", required=False)
parser.add_argument('-kernel_alpha','--kernel_alpha', type=float, default=1.0, help='param for information diffusion kernel', required=False)
parser.add_argument('-recon_alpha','--recon_alpha', type=float, default=-1.0, help='multiplier of the reconstruction loss when combined with mmd loss', required=False)
parser.add_argument('-recon_alpha_adapt','--recon_alpha_adapt', type=float, default=-1.0, help='adaptively change recon_alpha so that [total loss = mmd + recon_alpha_adapt * recon loss], set to -1 if no adapt', required=False)
parser.add_argument('-dropout_p','--dropout_p', type=float, default=-1.0, help='dropout probability in encoder', required=False)
parser.add_argument('-l2_alpha','--l2_alpha', type=float, default=-1.0, help='alpha multipler for L2 regularization on latent vector', required=False)
parser.add_argument('-latent_noise','--latent_noise', type=float, default=0.0, help='proportion of dirichlet noise added to the latent vector after softmax', required=False)
parser.add_argument('-topic_decoder_weight','--topic_decoder_weight', type=lambda x: (str(x).lower() == 'true'), default=False, help='extract topic words based on decoder weights or decoder outputs', required=False)
parser.add_argument('-retrain_enc_only','--retrain_enc_only', type=lambda x: (str(x).lower() == 'true'), default=False, help='only retrain the encoder for reconstruction loss', required=False)
parser.add_argument('-l2_alpha_retrain','--l2_alpha_retrain', type=float, default=0.1, help='alpha multipler for L2 regularization on encoder output during retraining', required=False)
args = vars(parser.parse_args())

In [5]:
import os
import numpy as np

In [6]:
from mxnet import gluon, io
import scipy.sparse as sparse
from sklearn.metrics import log_loss, v_measure_score

In [7]:
class Data(object):
    '''
    Data Generator object. Main functionality is contained in ``minibatch'' method
    and ``subsampled_labeled_data'' if training in a semi-supervised fashion.
    Introducing new datasets requires implementing ``load'' and possibly overwriting
    portions of ``__init__''.
    '''
    def __init__(self, batch_size=1, data_path='', ctx=mx.cpu(0)):
        '''
        Constructor for Data.
        Args
        ----
        batch_size: int, default 1
          An integer specifying the batch size - required for precompiling the graph.
        data_path: string, default ''
          This is primarily used by Mulan to specify which dataset to load from Mulan,
          e.g., data_path='bibtex'.
        ctx: mxnet device context, default mx.cpu(0)
          Which device to store/run the data and model on.
        Returns
        -------
        Data object
        '''
        self.batch_size = batch_size
        if data_path == '':
            data, labels, maps = self.load()
        else:
            data, labels, maps = self.load(data_path)
        self.ctx = ctx
        # # normalize the data:
        # def softmax(x):
        #     """Compute softmax values for each sets of scores in x."""
        #     e_x = np.exp(x - np.max(x, axis=1).reshape((-1,1)))
        #     return e_x / np.sum(e_x, axis=1).reshape((-1,1))
        # for i in range(len(data)):
        #     data[i] = softmax(data[i])

        data_names = ['train','valid','test','train_with_labels','valid_with_labels','test_with_labels']
        label_names = ['train_label', 'valid_label', 'test_label']

        self.data = dict(zip(data_names, data))
        self.labels = dict(zip(label_names, labels))

        # repeat data to at least match batch_size
        for k, v in self.data.items():
            if v is not None and v.shape[0] < self.batch_size:
                print('NOTE: Number of samples for {0} is smaller than batch_size ({1}<{2}). Duplicating samples to exceed batch_size.'.format(k,v.shape[0],self.batch_size))
                if type(v) is np.ndarray:
                    self.data[k] = np.tile(v, (self.batch_size // v.shape[0] + 1, 1))
                else:
                    self.data[k] = mx.nd.tile(v, (self.batch_size // v.shape[0] + 1, 1))

        for k, v in self.labels.items():
            if v is not None and v.shape[0] < self.batch_size:
                print('NOTE: Number of samples for {0} is smaller than batch_size ({1}<{2}). Duplicating samples to exceed batch_size.'.format(k,v.shape[0],self.batch_size))
                self.labels[k] = np.tile(v, (self.batch_size // v.shape[0] + 1, ))

        map_names = ['vocab2dim','dim2vocab','topic2dim','dim2topic']
        self.maps = dict(zip(map_names, maps))
        dls = [self.dataloader(d, batch_size) for d in data]
        dis = [iter(dl) if dl is not None else None for dl in dls]
        self.dataloaders = dict(zip(data_names, dls))
        self.dataiters = dict(zip(data_names, dis))
        self.wasreset = dict(zip(data_names, np.ones(len(data_names), dtype='bool')))

        self.data_dim = self.data['train'].shape[1]
        if self.data['train_with_labels'] is not None:
            self.label_dim = self.data['train_with_labels'].shape[1] - self.data['train'].shape[1]


    def dataloader(self, data, batch_size, shuffle=True):
        '''
        Constructs a data loader for generating minibatches of data.
        Args
        ----
        data: numpy array, no default
          The data from which to load minibatches.
        batch_size: integer, no default
          The # of samples returned in each minibatch.
        shuffle: boolean, default True
          Whether or not to shuffle the data prior to returning the data loader.
        Returns
        -------
        DataLoader: A gluon DataLoader iterator
        '''
        if data is None:
            return None
        else:
            # inds = np.arange(data.shape[0])
            # if shuffle:
            #     np.random.shuffle(inds)
            # ordered = data[inds]
            # N, r = divmod(data.shape[0], batch_size)
            # if r > 0:
            #     ordered = np.vstack([ordered, ordered[:r]])
            if type(data) is np.ndarray:
                return gluon.data.DataLoader(data, batch_size, last_batch='discard', shuffle=shuffle)
            else:
                return io.NDArrayIter(data={'data': data}, batch_size=batch_size, shuffle=shuffle, last_batch_handle='discard')

    def force_reset_data(self, key, shuffle=True):
        '''
        Resets minibatch index to zero to restart an epoch.
        Args
        ----
        key: string, no default
          Required to select appropriate data in ``data'' object,
          e.g., 'train', 'test', 'train_with_labels', 'test_with_labels'.
        shuffle: boolean, default True
          Whether or not to shuffle the data prior to returning the data loader.
        Returns
        -------
        Nothing.
        '''
        if self.data[key] is not None:
            if type(self.data[key]) is np.ndarray:
                self.dataloaders[key] = self.dataloader(self.data[key], self.batch_size, shuffle)
                self.dataiters[key] = iter(self.dataloaders[key])
            else:
                self.dataiters[key].hard_reset()
            self.wasreset[key] = True

    def minibatch(self, key, pad_width=0):
        '''
        Returns a minibatch of data (stored on device self.ctx).
        Args
        ----
        key: string, no default
          Required to select appropriate data in ``data'' object,
          e.g., 'train', 'test', 'train_with_labels', 'test_with_labels'.
        pad_width: integer, default 0
          The amount to zero-pad the labels to match the dimensionality of z.
        Returns
        -------
        minibatch: NDArray on device self.ctx
          An NDArray of size batch_size x # of features.
        '''
        if self.dataiters[key] is None:
            return None
        else:
            if type(self.data[key]) is np.ndarray:
                try:
                    mb = self.dataiters[key].__next__().reshape((self.batch_size, -1))
                    if pad_width > 0:
                        mb = mx.nd.concat(mb, mx.nd.zeros((self.batch_size, pad_width)))
                    return mb.copyto(self.ctx)
                except:
                    self.force_reset_data(key)
                    mb = self.dataiters[key].__next__().reshape((self.batch_size, -1))
                    if pad_width > 0:
                        mb = mx.nd.concat(mb, mx.nd.zeros((self.batch_size, pad_width)))
                    return mb.copyto(self.ctx)
            else:
                try:
                    mb = self.dataiters[key].__next__().data[0].as_in_context(self.ctx)
                    return mb
                except:
                    self.dataiters[key].hard_reset()
                    mb = self.dataiters[key].__next__().data[0].as_in_context(self.ctx)
                    return mb

    def get_documents(self, key, split_on=None):
        '''
        Retrieves a minibatch of documents via ``data'' object parameter.
        Args
        ----
        key: string, no default
          Required to select appropriate data in ``data'' object,
          e.g., 'train', 'test', 'train_with_labels', 'test_with_labels'.
        split_on: integer, default None
          Useful if self.data[key] contains both data and labels in one
          matrix and want to split them, e.g., split_on = data_dim.
        Returns
        -------
        minibatch: NDArray if split_on is None, else [NDarray, NDArray]
        '''
        if 'labels' in key:
            batch = self.minibatch(key, pad_width=self.label_pad_width)
        else:
            batch = self.minibatch(key)
        if split_on is not None:
            batch, labels = batch[:,:split_on], batch[:,split_on:]
            return batch, labels
        else:
            return batch

    @staticmethod
    def visualize_series(y, ylabel, file, args, iteration, total_samples, labels=None):
        '''
        Plots and saves a figure of y vs iterations and epochs to file.
        Args
        ----
        y: a list (of lists) or numpy array, no default
          A list (of possibly another list) of numbers to plot.
        ylabel: string, no default
          The label for the y-axis.
        file: string, no default
          A path with filename to save the figure to.
        args: dictionary, no default
          A dictionary of model, training, and evaluation specifications.
        iteration: integer, no default
          The current iteration in training.
        total_samples: integer, no default
          The total number of samples in the dataset - used along with batch_size
          to convert iterations to epochs.
        labels: list of strings, default None
          If y is a list of lists, the labels contains names for each element
          in the nested list. This is used to create an appropriate legend
          for the plot.
        Returns
        -------
        Nothing.
        '''
        if len(y) > 0:
            fig = plt.figure()
            ax = plt.subplot(111)
            x = np.linspace(0, iteration, num=len(y)) * args['batch_size'] / total_samples
            y = np.array(y)
            if len(y.shape) > 1:
                for i in range(y.shape[1]):
                    if labels is None:
                        plt.plot(x,y[:,i])
                    else:
                        plt.plot(x,y[:,i], label=labels[i])
            else:
                plt.plot(x,y)
            ax.set_ylabel(ylabel)
            ax.set_xlabel('Epochs')
            plt.grid(True)

            ax2 = ax.twiny()

            # https://pythonmatplotlibtips.blogspot.com/2018/01/add-second-x-axis-below-first-x-axis-python-matplotlib-pyplot.html
            # Decide the ticklabel position in the new x-axis,
            # then convert them to the position in the old x-axis
            # xticks list seems to be padded with extra lower and upper ticks --> subtract 2 from length
            newlabel = np.around(np.linspace(0, iteration, num=len(ax.get_xticks())-2)).astype('int') # labels of the xticklabels: the position in the new x-axis
            # ax2.set_xticks(ax.get_xticks())
            ax2.set_xticks(newlabel * args['batch_size'] / total_samples)
            ax2.set_xticklabels(newlabel//1000)

            ax2.xaxis.set_ticks_position('bottom') # set the position of the second x-axis to bottom
            ax2.xaxis.set_label_position('bottom') # set the position of the second x-axis to bottom
            ax2.spines['bottom'].set_position(('outward', 36))
            ax2.set_xlabel('Thousand Iterations')
            ax2.set_xlim(ax.get_xlim())

            if labels is not None:
                lgd = ax.legend(loc='center left', bbox_to_anchor=(1.05, 1))
                fig.savefig(args['saveto']+file, additional_artists=[lgd], bbox_inches='tight')
            else:
                fig.tight_layout()
                fig.savefig(args['saveto']+file)
            plt.close()

    def load(self, path=''):
        '''
        Loads data and maps from path.
        Args
        ----
        path: string, default ''
          A path to the data file.
        Returns
        -------
        data: list of numpy arrays
          A list of the different subsets of data, e.g.,
          `train', `test', 'train_with_labels', 'test_with_labels'.
        maps: list of dictionaries
          A list of dictionaries for mapping between dimensions and strings,
          e.g., 'vocab2dim', 'dim2vocab', 'topic2dim', 'dim2topic'.
        '''
        data = [np.empty((1,1)) for data in ['train','valid','test','train_with_labels','valid_with_labels','test_with_labels']]
        maps = [{'a':0}, {0:'a'}, {'Letters':0}, {0:'Letters'}]
        self.data_path = path + '***.npz'
        return data, maps

In [8]:
import nltk

In [9]:
class Wikitext103(Data):
    def __init__(self, batch_size, data_path='', ctx=None, saveto='', **kwargs):
        self.saveto = saveto
        super(Wikitext103, self).__init__(batch_size, data_path, ctx)

    def load(self, path='./data/wikitext-103', features='BoW', match_avitm=True):
        if path[:2] == '~/':
            path = os.path.join(os.path.expanduser(path[:2]), path[2:])

        ### Specify the file locations
        train_path = path + '/wikitext-103_tra.csr.npz'
        test_path = path + '/wikitext-103_test.csr.npz'
        vocab_path = path + '/vocab.txt'

        ### Load train
        train_csr = sparse.load_npz(train_path)
        train = np.array(train_csr.todense()).astype('float32')

        ### Load test
        test_csr = sparse.load_npz(test_path)
        test = np.array(test_csr.todense()).astype('float32')

        ### load vocab
        ENCODING = "ISO-8859-1"
        # ENCODING = "utf-8"
        with open(vocab_path, encoding=ENCODING) as f:
             vocab_list = [line.strip('\n') for line in f]

        # construct maps
        vocab2dim = dict(zip(vocab_list, range(len(vocab_list))))
        dim2vocab = reverse_dict(vocab2dim)

        return [train, None, test, None, None, None], [None, None, None], [vocab2dim, dim2vocab, None, None]

In [10]:
from scipy.special import logit, expit

from mxnet.gluon import nn

In [11]:
class ENet(gluon.HybridBlock):
    '''
    A gluon HybridBlock Encoder (skeleton) class.
    '''
    def __init__(self):
        '''
        Constructor for Encoder.
        Args
        ----
        None
        Returns
        -------
        Encoder object
        '''
        super(ENet, self).__init__()
            
    def hybrid_forward(self, x):
        '''
        Encodes x.
        Args
        ----
        x: mx.NDArray or sym, No default
          Input to encoder.
        Returns (should)
        -------
        params: list of NDArray or sym
          parameters for the encoding distribution
        samples: NDArray or sym
          samples drawn from the encoding distribution
        '''
        raise NotImplementedError('Need to write your own Encoder that inherits from ENet. Put this file in models/.')

    def init_weights(self, weights=None):
        '''
        Initializes the encoder weights. Default is Xavier initialization.
        Args
        ----
        weights: list of numpy arrays, No default
          Weights to load into the model. Not required. Preference is to
          load weights from file.
        Returns
        -------
        Nothing.
        '''
        loaded = False
        source = 'keyword argument'
        if self.weights_file != '' and weights is None:
            try:
                self.load_params(self.weights_file, self.model_ctx)
                source = 'mxnet weights file: '+self.weights_file
                print('NOTE: Loaded encoder weights from '+source+'.')
                if self.freeze:
                    self.freeze_params()
                    print('NOTE: Froze encoder weights from '+source+'.')
                weights = None
                loaded = True
            except:
                weights = pickle.load(open(self.weights_file,'rb'))
                source = 'pickle file: '+self.weights_file
        if weights is not None:
            assert self.n_layers == 0
            for p,w in zip(self.collect_params().values(), weights):
                if w is not None:
                    p.initialize(mx.init.Constant(mx.nd.array(w.squeeze())), ctx=self.model_ctx)
                    if self.freeze:
                        p.lr_mult = 0.
            print('NOTE: Loaded encoder weights from '+source+'.')
            if self.freeze:
                print('NOTE: Froze encoder weights from '+source+'.')
            loaded = True
        if not loaded:
            self.collect_params().initialize(mx.init.Xavier(), ctx=self.model_ctx)
            print('NOTE: Randomly initialized encoder weights.')
            # self.collect_params().initialize(mx.init.Zero(), ctx=self.model_ctx)
            # print('NOTE: initialized encoder weights to ZERO.')

    def freeze_params(self):
        for p in self.collect_params().values():
            p.lr_mult = 0.


class DNet(gluon.HybridBlock):
    '''
    A gluon HybridBlock Decoder (skeleton) class.
    '''
    def __init__(self):
        '''
        Constructor for Decoder.
        Args
        ----
        None
        Returns
        -------
        Decoder object
        '''
        super(DNet, self).__init__()

    def hybrid_forward(self, y, z):
        '''
        Decodes x.
        Args
        ----
        x: mx.NDArray or sym, no default
          Input to decoder.
        Returns (should)
        -------
        params: list of NDArray or sym
          parameters for the encoding distribution
        samples: NDArray or sym
          samples drawn from the encoding distribution. None if sampling is not implemented.
        '''
        raise NotImplementedError('Need to write your own Decoder that inherits from ENet. Put this file in models/.')

    def init_weights(self, weights=None):
        '''
        Initializes the decoder weights. Default is Xavier initialization.
        Args
        ----
        weights: list of numpy arrays, No default
          Weights to load into the model. Not required. Preference is to
          load weights from file.
        Returns
        -------
        Nothing.
        '''
        loaded = False
        source = 'keyword argument'
        if self.weights_file != '' and weights is None:
            try:
                self.load_params(self.weights_file, self.model_ctx)
                source = 'mxnet weights file: '+self.weights_file
                print('NOTE: Loaded decoder weights from '+source+'.')
                if self.freeze:
                    self.freeze_params()
                    print('NOTE: Froze decoder weights from '+source+'.')
                weights = None
                loaded = True
            except:
                weights = pickle.load(open(self.weights_file,'rb'))
                source = 'pickle file: '+self.weights_file
        if weights is not None:
            assert self.n_layers == 0
            for p,w in zip(self.collect_params().values(), weights):
                if w is not None:
                    p.initialize(mx.init.Constant(mx.nd.array(w.squeeze())), ctx=self.model_ctx)
                    if self.freeze:
                        p.lr_mult = 0.
            print('NOTE: Loaded decoder weights from '+source+'.')
            if self.freeze:
                print('NOTE: Froze decoder weights from '+source+'.')
            loaded = True
        if not loaded:
            self.collect_params().initialize(mx.init.Xavier(), ctx=self.model_ctx)
            print('NOTE: Randomly initialized decoder weights.')

    def freeze_params(self):
        for p in self.collect_params().values():
            p.lr_mult = 0.


class Compute(object):
    '''
    Skeleton class to manage training, testing, and retrieving outputs.
    See ``compute_op.py'' for ``flesh''.
    '''
    def __init__(self,  data, Enc, Dec,  Dis_y, args):
        '''
        Constructor for Compute.
        Returns
        -------
        Compute object
        '''
        self.data = data
        self.Enc = Enc
        self.Dec = Dec
        self.Dis_y = Dis_y
        self.args = args
        self.model_ctx = Enc.model_ctx
        self.ndim_y = args['ndim_y']

        weights_enc = Enc.collect_params()
        weights_dec = Dec.collect_params()
        weights_dis_y = Dis_y.collect_params()

        if self.args['optim'] == 'Adam':
            # args_dict = {'learning_rate': self.args['learning_rate'], 'beta1': self.args['betas'][0], 'beta2': self.args['betas'][1], 'epsilon': self.args['epsilon']}
            # optimizer_enc = gluon.Trainer(weights_enc, 'adam', args_dict)
            # optimizer_dec = gluon.Trainer(weights_dec, 'adam', args_dict)
            # optimizer_dis_y = gluon.Trainer(weights_dis_y, 'adam', args_dict)
            optimizer_enc = gluon.Trainer(weights_enc, 'adam', {'learning_rate': self.args['learning_rate'], 'beta1': 0.99})
            optimizer_dec = gluon.Trainer(weights_dec, 'adam', {'learning_rate': self.args['learning_rate'], 'beta1': 0.99})
            optimizer_dis_y = gluon.Trainer(weights_dis_y, 'adam', {'learning_rate': self.args['learning_rate']})
        if self.args['optim'] == 'Adadelta':
            # note: learning rate has no effect on Adadelta --> https://mxnet.incubator.apache.org/_modules/mxnet/optimizer.html#AdaDelta
            args_dict = {'rescale_grad': 1}  #, 'clip_gradient': 0.1}
            optimizer_enc = gluon.Trainer(weights_enc, 'adadelta', args_dict)
            optimizer_dec = gluon.Trainer(weights_dec, 'adadelta', args_dict)
            optimizer_dis_y = gluon.Trainer(weights_dis_y, 'adadelta', args_dict)
        if self.args['optim'] == 'RMSprop':
            args_dict = {'learning_rate': self.args['learning_rate'], 'epsilon': 1e-10, 'alpha': 0.9}
            optimizer_enc = gluon.Trainer(weights_enc, 'rmsprop', args_dict)
            optimizer_dec = gluon.Trainer(weights_dec, 'rmsprop', args_dict)
            optimizer_dis_y = gluon.Trainer(weights_dis_y, 'rmsprop', args_dict)
        if self.args['optim'] == 'SGD':
            args_dict = {'learning_rate': self.args['learning_rate'], 'wd': self.args['weight_decay'], 'rescale_grad': 1., 'momentum': 0.0, 'lazy_update': False}
            optimizer_enc = gluon.Trainer(weights_enc, 'sgd', args_dict)
            optimizer_dec = gluon.Trainer(weights_dec, 'sgd', args_dict)
            optimizer_dis_y = gluon.Trainer(weights_dis_y, 'sgd', args_dict)

        self.optimizer_enc = optimizer_enc
        self.optimizer_dec = optimizer_dec
        self.optimizer_dis_y = optimizer_dis_y
        self.weights_enc = weights_enc
        self.weights_dec = weights_dec
        self.weights_dis_y = weights_dis_y

    def train_op(self):
        '''
        Trains the model using one minibatch of data.
        '''
        return None, None, None, None

    def test_op(self, num_samples=None, num_epochs=None, reset=True, dataset='test'):
        '''
        Evaluates the model using num_samples.
        Args
        ----
        num_samples: integer, default None
          The number of samples to evaluate on. This is converted to
          evaluating on (num_samples // batch_size) minibatches.
        num_epochs: integer, default None
          The number of epochs to evaluate on. This used if num_samples
          is not specified. If neither is specified, defaults to 1 epoch.
        reset: bool, default True
          Whether to reset the test data index to 0 before iterating
          through and evaluating on minibatches.
        dataset: string, default 'test':
          Which dataset to evaluate on: 'valid' or 'test'.
        '''
        if num_samples is None:
            num_samples = self.data.data[dataset].shape[0]

        if reset:
            # Reset Data to Index Zero
            self.data.force_reset_data(dataset)
            self.data.force_reset_data(dataset+'_with_labels')

        return None, None, None, None

    def get_outputs(self, num_samples=None, num_epochs=None, reset=True, dataset='test'):
        '''
        Retrieves raw outputs from model for num_samples.
        Args
        ----
        num_samples: integer, default None
          The number of samples to evaluate on. This is converted to
          evaluating on (num_samples // batch_size) minibatches.
        num_epochs: integer, default None
          The number of epochs to evaluate on. This used if num_samples
          is not specified. If neither is specified, defaults to 1 epoch.
        reset: bool, default True
          Whether to reset the test data index to 0 before iterating
          through and evaluating on minibatches.
        dataset: string, default 'test':
          Which dataset to evaluate on: 'valid' or 'test'.
        '''
        if num_samples is None:
            num_samples = self.data.data[dataset].shape[0]

        if reset:
            # Reset Data to Index Zero
            self.data.force_reset_data(dataset)
            self.data.force_reset_data(dataset+'_with_labels')

        return None, None, None, None, None, None

In [12]:
class Encoder(ENet):
    '''
    A gluon HybridBlock Encoder class
    '''
    def __init__(self, model_ctx, batch_size, input_dim, n_hidden=64, ndim_y=16, ndim_z=10, n_layers=0, nonlin=None,
                 weights_file='', freeze=False, latent_nonlin='sigmoid', **kwargs):
        '''
        Constructor for encoder.
        Args
        ----
        model_ctx: mxnet device context, No default
          Which device to store/run the data and model on.
        batch_size: integer, No default
          The minibatch size.
        input_dim: integer, No default
          The data dimensionality that is input to the encoder.
        n_hidden: integer or list, default 64
          If integer, specifies the number of hidden units in
          every hidden layer.
          If list, each element specifies the number of hidden
          units in each hidden layer.
        output_dim: integer, default 10
          The dimensionality of the latent space, z.
        n_layers: integer, default 0
          The number of hidden layers.
        nonlin: string, default None
          The nonlinearity to use in every hidden layer.
        weights_file: string, default ''
          The path to the file (mxnet params file or pickle file)
          containing weights for each layer of the encoder.
        freeze: boolean, default False
          Whether to freeze the encoder weights (MIGHT BE BROKEN).
        latent_nonlin: string, default 'sigmoid'
          Which space to use for the latent variable:
            if 'sigmoid': z in (0,1)
            else: z in (-inf,inf)
        Parameters
        ----------
        Returns
        -------
        encoder object.
        '''
        super(Encoder, self).__init__()

        if n_layers >= 0:
            if isinstance(n_hidden, list):
                n_hidden = n_hidden[0]
                print('NOTE: Encoder ignoring list of hiddens because n_layer >= 0. Just using first element.')
            n_hidden = n_layers*[n_hidden]
        else:
            n_layers = len(n_hidden)
            print('NOTE: Encoder reading n_hidden as list.')

        if nonlin == '':
            nonlin = None
        
        in_units = input_dim
        with self.name_scope():
            self.main = nn.HybridSequential(prefix='encoder')
            for i in range(n_layers):
                self.main.add(nn.Dense(n_hidden[i], in_units=in_units, activation=nonlin))
                in_units = n_hidden[i]
            self.main.add(nn.Dense(ndim_y, in_units=in_units, activation=None))

        self.model_ctx = model_ctx
        self.input_dim = input_dim
        self.n_hidden = n_hidden
        self.ndim_y = ndim_y
        self.ndim_z = ndim_z
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.nonlin = nonlin
        self.latent_nonlin = latent_nonlin
        self.weights_file = weights_file
        self.freeze = freeze
        self.dist_params = [None]

    def hybrid_forward(self, F, x):
        '''
        Passes the input through the encoder.
        Args
        ----
        F: mxnet.nd or mxnet.sym, No default
          This will be passed implicitly when calling hybrid forward.
        x: NDarray or mxnet symbol, No default
          The input to the encoder.
        Returns
        -------
        dist_params: list
          A list of the posterior parameters as NDarrays, each being of size batch_size x z_dim.
        samples: NDarray
          The posterior samples as a batch_size x z_dim NDarray.
        '''

        y = self.main(x)

        return y


class Decoder(DNet):
    '''
    A gluon HybridBlock Decoder class with Multinomial likelihood, p(x|z).
    '''
    def __init__(self, model_ctx, batch_size, output_dim, ndim_y=16,  n_hidden=64, n_layers=0, nonlin='',
                 weights_file='', freeze=False, latent_nonlin='', **kwargs):
        '''
        Constructor for Multinomial decoder.
        Args
        ----
        model_ctx: mxnet device context, No default
          Which device to store/run the data and model on.
        batch_size: integer, No default
          The minibatch size.
        n_hidden: integer or list, default 64
          If integer, specifies the number of hidden units in
          every hidden layer.
          If list, each element specifies the number of hidden
          units in each hidden layer.
        output_dim: integer, No default
          The dimensionality of the latent space, z.
        n_layers: integer, default 0
          The number of hidden layers.
        nonlin: string, default 'sigmoid'
          The nonlinearity to use in every hidden layer.
        weights_file: string, default ''
          The path to the file (mxnet params file or pickle file)
          containing weights for each layer of the encoder.
        freeze: boolean, default False
          Whether to freeze the encoder weights (MIGHT BE BROKEN).
        latent_nonlin: string, default 'sigmoid'
          Which space to use for the latent variable:
            if 'sigmoid': z in (0,1)
            else: z in (-inf,inf)
        Parameters
        ----------
        Returns
        -------
        Multinomial decoder object.
        '''
        super(Decoder, self).__init__()

        if n_layers >= 0:
            if isinstance(n_hidden, list):
                n_hidden = n_hidden[0]
                print('NOTE: Decoder ignoring list of hiddens because n_layer >= 0. Just using first element.')
            n_hidden = n_layers*[n_hidden]
        else:
            n_layers = len(n_hidden)
            print('NOTE: Decoder reading n_hidden as list.')

        if nonlin == '':
            nonlin = None

        in_units = n_hidden[0]
        with self.name_scope():
            self.main = nn.HybridSequential(prefix='decoder')
            self.main.add(nn.Dense(n_hidden[0], in_units=ndim_y, activation=None))

        self.model_ctx = model_ctx
        self.batch_size = batch_size
        self.ndim_y = ndim_y
        self.n_hidden = n_hidden
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.nonlin = nonlin
        self.latent_nonlin = latent_nonlin
        self.weights_file = weights_file
        self.freeze = freeze

    def hybrid_forward(self, F, y):
        '''
        Passes the input through the decoder.
        Args
        ----
        F: mxnet.nd or mxnet.sym, No default
          This will be passed implicitly when calling hybrid forward.
        x: NDarray or mxnet symbol, No default
          The input to the decoder.
        Returns
        -------
        dist_params: list
          A list of the multinomial parameters as NDarrays, each being of size batch_size x z_dim.
        samples: NDarray
          The multinomial samples as a batch_size x z_dim NDarray (NOT IMPLEMENTED).
        '''
        out = self.main(y)
        return out

    def y_as_topics(self, eps=1e-10):
        y = np.eye(self.ndim_y)
        return mx.nd.array(y)

class Discriminator_y(ENet):
    '''
    A gluon HybridBlock Discriminator Class for y
    '''

    def __init__(self, model_ctx, batch_size, output_dim=2, ndim_y=16, n_hidden=64, n_layers=0, nonlin='sigmoid',
                 weights_file='', freeze=False, latent_nonlin='sigmoid', apply_softmax=False, **kwargs):
        '''
        Constructor for Discriminator Class for y.
        Args
        ----
        model_ctx: mxnet device context, No default
          Which device to store/run the data and model on.
        batch_size: integer, No default
          The minibatch size.
        n_hidden: integer or list, default 64
          If integer, specifies the number of hidden units in
          every hidden layer.
          If list, each element specifies the number of hidden
          units in each hidden layer.
        output_dim: integer, No default
          The dimensionality of the latent space, z.
        n_layers: integer, default 0
          The number of hidden layers.
        nonlin: string, default 'sigmoid'
          The nonlinearity to use in every hidden layer.
        weights_file: string, default ''
          The path to the file (mxnet params file or pickle file)
          containing weights for each layer of the encoder.
        freeze: boolean, default False
          Whether to freeze the encoder weights (MIGHT BE BROKEN).
        latent_nonlin: string, default 'sigmoid'
          Which space to use for the latent variable:
            if 'sigmoid': z in (0,1)
            else: z in (-inf,inf)
        Parameters
        ----------
        Returns
        -------
        Multinomial Discriminator object.
        '''
        super(Discriminator_y, self).__init__()

        if n_layers >= 0:
            if isinstance(n_hidden, list):
                n_hidden = n_hidden[0]
                print('NOTE: Decoder ignoring list of hiddens because n_layer >= 0. Just using first element.')
            n_hidden = n_layers * [n_hidden]
        else:
            n_layers = len(n_hidden)
            print('NOTE: Decoder reading n_hidden as list.')

        if latent_nonlin != 'sigmoid':
            print('NOTE: Latent z will be fed to decoder in logit-space (-inf,inf).')
        else:
            print('NOTE: Latent z will be fed to decoder in probability-space (0,1).')

        if nonlin == '':
            nonlin = None

        in_units = ndim_y
        with self.name_scope():
            self.main = nn.HybridSequential(prefix='discriminator_y')
            for i in range(n_layers):
                self.main.add(nn.Dense(n_hidden[i], in_units=in_units, activation=nonlin))
                in_units = n_hidden[i]
            self.main.add(nn.Dense(output_dim, in_units=in_units, activation=None))

        self.model_ctx = model_ctx
        self.batch_size = batch_size
        self.ndim_y = ndim_y
        self.n_hidden = n_hidden
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.nonlin = nonlin
        self.latent_nonlin = latent_nonlin
        self.weights_file = weights_file
        self.freeze = freeze
        self.apply_softmax = apply_softmax

    def hybrid_forward(self, F, y):
        '''
        Passes the input through the decoder.
        Args
        ----
        F: mxnet.nd or mxnet.sym, No default
          This will be passed implicitly when calling hybrid forward.
        x: NDarray or mxnet symbol, No default
          The input to the decoder.
        '''
        logit = self.main(y)
        if self.apply_softmax:
            return F.softmax(logit)
        return logit

In [13]:
def mmd_loss(x, y, ctx_model, t=0.1, kernel='diffusion'):
    '''
    computes the mmd loss with information diffusion kernel
    :param x: batch_size x latent dimension
    :param y:
    :param t:
    :return:
    '''
    eps = 1e-6
    n,d = x.shape
    if kernel == 'tv':
        sum_xx = nd.zeros(1, ctx=ctx_model)
        for i in range(n):
            for j in range(i+1, n):
                sum_xx = sum_xx + nd.norm(x[i] - x[j], ord=1)
        sum_xx = sum_xx / (n * (n-1))

        sum_yy = nd.zeros(1, ctx=ctx_model)
        for i in range(y.shape[0]):
            for j in range(i+1, y.shape[0]):
                sum_yy = sum_yy + nd.norm(y[i] - y[j], ord=1)
        sum_yy = sum_yy / (y.shape[0] * (y.shape[0]-1))

        sum_xy = nd.zeros(1, ctx=ctx_model)
        for i in range(n):
            for j in range(y.shape[0]):
                sum_xy = sum_xy + nd.norm(x[i] - y[j], ord=1)
        sum_yy = sum_yy / (n * y.shape[0])
    else:
        qx = nd.sqrt(nd.clip(x, eps, 1))
        qy = nd.sqrt(nd.clip(y, eps, 1))
        xx = nd.dot(qx, qx, transpose_b=True)
        yy = nd.dot(qy, qy, transpose_b=True)
        xy = nd.dot(qx, qy, transpose_b=True)

        def diffusion_kernel(a, tmpt, dim):
            # return (4 * np.pi * tmpt)**(-dim / 2) * nd.exp(- nd.square(nd.arccos(a)) / tmpt)
            return nd.exp(- nd.square(nd.arccos(a)) / tmpt)

        off_diag = 1 - nd.eye(n, ctx=ctx_model)
        k_xx = diffusion_kernel(nd.clip(xx, 0, 1-eps), t, d-1)
        k_yy = diffusion_kernel(nd.clip(yy, 0, 1-eps), t, d-1)
        k_xy = diffusion_kernel(nd.clip(xy, 0, 1-eps), t, d-1)
        sum_xx = (k_xx * off_diag).sum() / (n * (n-1))
        sum_yy = (k_yy * off_diag).sum() / (n * (n-1))
        sum_xy = 2 * k_xy.sum() / (n * n)
    return sum_xx + sum_yy - sum_xy


class Unsupervised(Compute):
    '''
    Class to manage training, testing, and
    retrieving outputs.
    '''
    def __init__(self, data, Enc, Dec,  Dis_y, args):
        '''
        Constructor.
        Args
        ----
        Returns
        -------
        Compute object
        '''
        super(Unsupervised, self).__init__(data, Enc, Dec, Dis_y, args)

    def unlabeled_train_op_mmd_combine(self, update_enc=True):
        '''
        Trains the MMD model
        '''
        batch_size = self.args['batch_size']
        model_ctx = self.model_ctx
        eps = 1e-10

        # Retrieve data
        docs = self.data.get_documents(key='train')

        y_true = np.random.dirichlet(np.ones(self.ndim_y) * self.args['dirich_alpha'], size=batch_size)
        y_true = nd.array(y_true, ctx=model_ctx)

        with autograd.record():
            ### reconstruction phase ###
            y_onehot_u = self.Enc(docs)
            y_onehot_u_softmax = nd.softmax(y_onehot_u)
            if self.args['latent_noise'] > 0:
                y_noise = np.random.dirichlet(np.ones(self.ndim_y) * self.args['dirich_alpha'], size=batch_size)
                y_noise = nd.array(y_noise, ctx=model_ctx)
                y_onehot_u_softmax = (1 - self.args['latent_noise']) * y_onehot_u_softmax + self.args['latent_noise'] * y_noise
            x_reconstruction_u = self.Dec(y_onehot_u_softmax)

            logits = nd.log_softmax(x_reconstruction_u)
            loss_reconstruction = nd.mean(nd.sum(- docs * logits, axis=1))
            loss_total = loss_reconstruction * self.args['recon_alpha']

            ### mmd phase ###
            if self.args['adverse']:
                y_fake = self.Enc(docs)
                y_fake = nd.softmax(y_fake)
                loss_mmd = mmd_loss(y_true, y_fake, ctx_model=model_ctx, t=self.args['kernel_alpha'])
                loss_total = loss_total + loss_mmd

            if self.args['l2_alpha'] > 0:
                loss_total = loss_total + self.args['l2_alpha'] * nd.mean(nd.sum(nd.square(y_onehot_u), axis=1))

            loss_total.backward()

        self.optimizer_enc.step(1)
        self.optimizer_dec.step(1)  # self.m.args['batch_size']

        latent_max = nd.zeros(self.args['ndim_y'], ctx=model_ctx)
        for max_ind in nd.argmax(y_onehot_u, axis=1):
            latent_max[max_ind] += 1.0
        latent_max /= batch_size
        latent_entropy = nd.mean(nd.sum(- y_onehot_u_softmax * nd.log(y_onehot_u_softmax + eps), axis=1))
        latent_v = nd.mean(y_onehot_u_softmax, axis=0)
        dirich_entropy = nd.mean(nd.sum(- y_true * nd.log(y_true + eps), axis=1))

        if self.args['adverse']:
            loss_mmd_return = loss_mmd.asscalar()
        else:
            loss_mmd_return = 0.0
        return nd.mean(loss_reconstruction).asscalar(), loss_mmd_return, latent_max.asnumpy(), latent_entropy.asscalar(), latent_v.asnumpy(), dirich_entropy.asscalar()


    def retrain_enc(self, l2_alpha=0.1):
        docs = self.data.get_documents(key='train')
        with autograd.record():
            ### reconstruction phase ###
            y_onehot_u = self.Enc(docs)
            y_onehot_u_softmax = nd.softmax(y_onehot_u)
            x_reconstruction_u = self.Dec(y_onehot_u_softmax)

            logits = nd.log_softmax(x_reconstruction_u)
            loss_reconstruction = nd.mean(nd.sum(- docs * logits, axis=1))
            loss_reconstruction = loss_reconstruction + l2_alpha * nd.mean(nd.norm(y_onehot_u, ord=1, axis=1))
            loss_reconstruction.backward()

        self.optimizer_enc.step(1)
        return loss_reconstruction.asscalar()


    def unlabeled_train_op_adv_combine_add(self, update_enc=True):
        '''
        Trains the GAN model
        '''
        batch_size = self.args['batch_size']
        model_ctx = self.model_ctx
        eps = 1e-10
        ##########################
        ### unsupervised phase ###
        ##########################
        # Retrieve data
        docs = self.data.get_documents(key='train')

        class_true = nd.zeros(batch_size, dtype='int32', ctx=model_ctx)
        class_fake = nd.ones(batch_size, dtype='int32', ctx=model_ctx)
        loss_reconstruction = nd.zeros((1,), ctx=model_ctx)

        ### adversarial phase ###
        discriminator_z_confidence_true = nd.zeros(shape=(1,), ctx=model_ctx)
        discriminator_z_confidence_fake = nd.zeros(shape=(1,), ctx=model_ctx)
        discriminator_y_confidence_true = nd.zeros(shape=(1,), ctx=model_ctx)
        discriminator_y_confidence_fake = nd.zeros(shape=(1,), ctx=model_ctx)
        loss_discriminator = nd.zeros(shape=(1,), ctx=model_ctx)
        dirich_entropy = nd.zeros(shape=(1,), ctx=model_ctx)

        ### generator phase ###
        loss_generator = nd.zeros(shape=(1,), ctx=model_ctx)

        ### reconstruction phase ###
        with autograd.record():
            y_u = self.Enc(docs)
            y_onehot_u_softmax = nd.softmax(y_u)
            x_reconstruction_u = self.Dec(y_onehot_u_softmax)

            logits = nd.log_softmax(x_reconstruction_u)
            loss_reconstruction = nd.sum(- docs * logits, axis=1)
            loss_total = loss_reconstruction * self.args['recon_alpha']

            if self.args['adverse']: #and np.random.rand()<0.8:
                y_true = np.random.dirichlet(np.ones(self.ndim_y) * self.args['dirich_alpha'], size=batch_size)
                y_true = nd.array(y_true, ctx=model_ctx)
                dy_true = self.Dis_y(y_true)
                dy_fake = self.Dis_y(y_onehot_u_softmax)
                discriminator_y_confidence_true = nd.mean(nd.softmax(dy_true)[:, 0])
                discriminator_y_confidence_fake = nd.mean(nd.softmax(dy_fake)[:, 1])
                softmaxCEL = gluon.loss.SoftmaxCrossEntropyLoss()
                loss_discriminator = softmaxCEL(dy_true, class_true) + \
                                       softmaxCEL(dy_fake, class_fake)
                loss_generator = softmaxCEL(dy_fake, class_true)
                loss_total = loss_total + loss_discriminator + loss_generator
                dirich_entropy = nd.mean(nd.sum(- y_true * nd.log(y_true + eps), axis=1))

        loss_total.backward()

        self.optimizer_enc.step(batch_size)
        self.optimizer_dec.step(batch_size)
        self.optimizer_dis_y.step(batch_size)

        latent_max = nd.zeros(self.args['ndim_y'], ctx=model_ctx)
        for max_ind in nd.argmax(y_onehot_u_softmax, axis=1):
            latent_max[max_ind] += 1.0
        latent_max /= batch_size
        latent_entropy = nd.mean(nd.sum(- y_onehot_u_softmax * nd.log(y_onehot_u_softmax + eps), axis=1))
        latent_v = nd.mean(y_onehot_u_softmax, axis=0)

        return nd.mean(loss_discriminator).asscalar(), nd.mean(loss_generator).asscalar(), nd.mean(loss_reconstruction).asscalar(), \
               nd.mean(discriminator_z_confidence_true).asscalar(), nd.mean(discriminator_z_confidence_fake).asscalar(), \
               nd.mean(discriminator_y_confidence_true).asscalar(), nd.mean(discriminator_y_confidence_fake).asscalar(), \
               latent_max.asnumpy(), latent_entropy.asscalar(), latent_v.asnumpy(), dirich_entropy.asscalar()


    def test_synthetic_op(self):
        batch_size = self.args['batch_size']
        dataset = 'train'
        num_samps = self.data.data[dataset].shape[0]
        batches = int(np.ceil(num_samps / batch_size))
        batch_iter = range(batches)
        enc_out = nd.zeros(shape=(batches * batch_size, self.ndim_y))
        for batch in batch_iter:
            # 1. Retrieve data
            if self.args['data_source'] == 'Ian':
                docs = self.data.get_documents(key=dataset)
            # 2. Compute loss
            y_onehot_u = self.Enc(docs)
            y_onehot_softmax = nd.softmax(y_onehot_u)
            enc_out[batch*batch_size:(batch+1)*batch_size, :] = y_onehot_softmax

        return enc_out

    def test_op(self, num_samples=None, num_epochs=None, reset=True, dataset='test'):
        '''
        Evaluates the model using num_samples.
        Args
        ----
        num_samples: integer, default None
          The number of samples to evaluate on. This is converted to
          evaluating on (num_samples // batch_size) minibatches.
        num_epochs: integer, default None
          The number of epochs to evaluate on. This used if num_samples
          is not specified. If neither is specified, defaults to 1 epoch.
        reset: bool, default True
          Whether to reset the test data index to 0 before iterating
          through and evaluating on minibatches.
        dataset: string, default 'test':
          Which dataset to evaluate on: 'valid' or 'test'.
        Returns
        -------
        Loss_u: float
          The loss on the unlabeled data.
        Loss_l: float
          The loss on the labeled data.
        Eval_u: list of floats
          A list of evaluation metrics on the unlabeled data.
        Eval_l: list of floats
          A list of evaluation metrics on the labeled data.
        '''
        batch_size = self.args['batch_size']
        model_ctx = self.model_ctx

        if num_samples is None and num_epochs is None:
            # assume full dataset evaluation
            num_epochs = 1

        if reset:
            # Reset Data to Index Zero
            if self.data.data[dataset] is not None:
                self.data.force_reset_data(dataset)
            if self.data.data[dataset + '_with_labels'] is not None:
                self.data.force_reset_data(dataset+'_with_labels')

        # Unlabeled Data
        u_loss = 'NA'
        u_eval = []
        if self.data.data[dataset] is not None:
            u_loss = 0
            if num_samples is None:
                num_samps = self.data.data[dataset].shape[0] * num_epochs
            else:
                num_samps = num_samples
            batches = int(np.ceil(num_samps / self.args['batch_size']))
            batch_iter = range(batches)
            if batches > 1: batch_iter = tqdm(batch_iter, desc='unlabeled')
            for batch in batch_iter:
                # 1. Retrieve data
                docs = self.data.get_documents(key=dataset)

                # 2. Compute loss
                y_u = self.Enc(docs)
                y_onehot_u_softmax = nd.softmax(y_u)
                x_reconstruction_u = self.Dec(y_onehot_u_softmax)

                logits = nd.log_softmax(x_reconstruction_u)
                loss_recon_unlabel = nd.sum(- docs * logits, axis=1)

                # 3. Convert to numpy
                u_loss += nd.mean(loss_recon_unlabel).asscalar()
            u_loss /= batches

        # Labeled Data
        l_loss = 0.0
        l_acc = 0.0
        if self.data.data[dataset+'_with_labels'] is not None:
            l_loss = 0
            if num_samples is None:
                num_samps = self.data.data[dataset+'_with_labels'].shape[0] * num_epochs
            else:
                num_samps = num_samples
            batches = int(np.ceil(num_samps / self.args['batch_size']))
            batch_iter = range(batches)
            if batches > 1: batch_iter = tqdm(batch_iter, desc='labeled')
            softmaxCEL = gluon.loss.SoftmaxCrossEntropyLoss(sparse_label=False)
            for batch in batch_iter:
                # 1. Retrieve data
                labeled_docs, labels = self.data.get_documents(key=dataset+'_with_labels', split_on=self.data.data_dim)
                # 2. Compute loss
                y_u = self.Enc(docs)
                y_onehot_u_softmax = nd.softmax(y_u)
                class_pred = nd.argmax(y_onehot_u_softmax, axis=1)
                l_a = labels[list(range(labels.shape[0])), class_pred]
                l_acc += nd.mean(l_a).asscalar()
                labels = labels / nd.sum(labels, axis=1, keepdims=True)
                l_l = softmaxCEL(y_onehot_u_softmax, labels)

                # 3. Convert to numpy
                l_loss += nd.mean(l_l).asscalar()
            l_loss /= batches
            l_acc /= batches

        return u_loss, l_loss, l_acc


    def save_latent(self, saveto):
        before_softmax = True
        try:
            if type(self.data.data['train']) is np.ndarray:
                dataset_train = gluon.data.dataset.ArrayDataset(self.data.data['train'])
                train_data = gluon.data.DataLoader(dataset_train, self.args['batch_size'], shuffle=False, last_batch='discard')

                dataset_val = gluon.data.dataset.ArrayDataset(self.data.data['valid'])
                val_data = gluon.data.DataLoader(dataset_val, self.args['batch_size'], shuffle=False, last_batch='discard')

                dataset_test = gluon.data.dataset.ArrayDataset(self.data.data['test'])
                test_data = gluon.data.DataLoader(dataset_test, self.args['batch_size'], shuffle=False, last_batch='discard')
            else:
                train_data = io.NDArrayIter(data={'data': self.data.data['train']}, batch_size=self.args['batch_size'],
                                            shuffle=False, last_batch_handle='discard')
                val_data = io.NDArrayIter(data={'data': self.data.data['valid']}, batch_size=self.args['batch_size'],
                                            shuffle=False, last_batch_handle='discard')
                test_data = io.NDArrayIter(data={'data': self.data.data['test']}, batch_size=self.args['batch_size'],
                                            shuffle=False, last_batch_handle='discard')
        except:
            print("Loading error during save_latent. Probably caused by not having validation or test set!")
            return

        train_output = np.zeros((self.data.data['train'].shape[0], self.ndim_y))
        # train_label_output = np.zeros(self.data.data['train'].shape[0])
        # for i, (data, label) in enumerate(train_data):
        for i, data in enumerate(train_data):
            if type(data) is io.DataBatch:
                data = data.data[0].as_in_context(self.model_ctx)
            else:
                data = data.as_in_context(self.model_ctx)
            if before_softmax:
                output = self.Enc(data)
            else:
                output = nd.softmax(self.Enc(data))
            train_output[i*self.args['batch_size']:(i+1)*self.args['batch_size']] = output.asnumpy()
            # train_label_output[i*self.args['batch_size']:(i+1)*self.args['batch_size']] = label.asnumpy()
        train_output = np.delete(train_output, np.s_[(i+1)*self.args['batch_size']:], 0)
        # train_label_output = np.delete(train_label_output, np.s_[(i+1)*self.args['batch_size']:])
        np.save(os.path.join(saveto, self.args['domain']+'train_latent.npy'), train_output)
        # np.save(os.path.join(saveto, self.args['domain']+'train_latent_label.npy'), train_label_output)

        val_output = np.zeros((self.data.data['valid'].shape[0], self.ndim_y))
        # train_label_output = np.zeros(self.data.data['train'].shape[0])
        # for i, (data, label) in enumerate(train_data):
        for i, data in enumerate(val_data):
            if type(data) is io.DataBatch:
                data = data.data[0].as_in_context(self.model_ctx)
            else:
                data = data.as_in_context(self.model_ctx)
            if before_softmax:
                output = self.Enc(data)
            else:
                output = nd.softmax(self.Enc(data))
            val_output[i*self.args['batch_size']:(i+1)*self.args['batch_size']] = output.asnumpy()
            # train_label_output[i*self.args['batch_size']:(i+1)*self.args['batch_size']] = label.asnumpy()
        val_output = np.delete(val_output, np.s_[(i+1)*self.args['batch_size']:], 0)
        # train_label_output = np.delete(train_label_output, np.s_[(i+1)*self.args['batch_size']:])
        np.save(os.path.join(saveto, self.args['domain']+'val_latent.npy'), val_output)
        # np.save(os.path.join(saveto, self.args['domain']+'train_latent_label.npy'), train_label_output)

        test_output = np.zeros((self.data.data['test'].shape[0], self.ndim_y))
        # test_label_output = np.zeros(self.data.data['test'].shape[0])
        # for i, (data, label) in enumerate(test_data):
        for i, data in enumerate(test_data):
            if type(data) is io.DataBatch:
                data = data.data[0].as_in_context(self.model_ctx)
            else:
                data = data.as_in_context(self.model_ctx)
            if before_softmax:
                output = self.Enc(data)
            else:
                output = nd.softmax(self.Enc(data))
            test_output[i*self.args['batch_size']:(i+1)*self.args['batch_size']] = output.asnumpy()
            # test_label_output[i*self.args['batch_size']:(i+1)*self.args['batch_size']] = label.asnumpy()
        test_output = np.delete(test_output, np.s_[(i+1)*self.args['batch_size']:], 0)
        # test_label_output = np.delete(test_label_output, np.s_[(i+1)*self.args['batch_size']:])
        np.save(os.path.join(saveto, self.args['domain']+'test_latent.npy'), test_output)
        # np.save(os.path.join(saveto, self.args['domain']+'test_latent_label.npy'), test_label_output)

In [14]:
args['latent_noise'] >= 0.0 and args['latent_noise'] <= 1

True

In [15]:
if args['description'] == '':
    args['description'] = args['domain'] + '-' + args['algorithm'] + '-' + args['model']
    if args['un_label_coeffs'][0] > 0 and args['un_label_coeffs'][1] == 0:
        args['description'] += '-unsup'
    elif args['un_label_coeffs'][0] > 0 and args['un_label_coeffs'][1] > 0:
        args['description'] += '-semisup'
    else:
        args['description'] += '-sup'
elif args['description'].isdigit():
    args['description'] = args['domain'] + '-' + args['algorithm'] + '-' + args['model'] + '-' + args['description']

KeyError: 'un_label_coeffs'

In [17]:
model_ctx = gpu_helper(0)
model_ctx

gpu(0)

In [18]:
data = Wikitext103(batch_size=256, data_path='./data/wikitext-103', ctx=model_ctx, saveto='./')

NOTE: Number of samples for test is smaller than batch_size (60<256). Duplicating samples to exceed batch_size.


In [19]:
print('train dimension = ', data.data['train'].shape)

train dimension =  (28472, 20000)


In [24]:
if type(data.data['train']) is np.ndarray:
    mean_length = np.mean(np.sum(data.data['train'], axis=1))
else:
    mean_length = mx.nd.mean(mx.nd.sum(data.data['train'], axis=1)).asscalar()
mean_length

1392.214

In [25]:
vocab_size = data.data['train'].shape[1]
vocab_size

20000

In [27]:
if data.data['train_with_labels'] is not None:
    print('train_with_labels dimension = ', data.data['train_with_labels'].shape)

In [29]:
if args['recon_alpha'] < 0:
    args['recon_alpha'] = 1.0 / (mean_length * np.log(vocab_size))
args['recon_alpha']

7.252802419767204e-05

In [30]:
Enc = Encoder(model_ctx=model_ctx, batch_size=args['batch_size'], input_dim=args['ndim_x'], ndim_y=args['ndim_y'],
              n_hidden=args['enc_n_hidden'], n_layers=args['enc_n_layer'], nonlin=args['enc_nonlinearity'],
              weights_file=args['enc_weights'], freeze=args['enc_freeze'], latent_nonlin=args['latent_nonlinearity'])
Dec = Decoder(model_ctx=model_ctx, batch_size=args['batch_size'], output_dim=args['ndim_x'], ndim_y=args['ndim_y'],
              n_hidden=args['dec_n_hidden'], n_layers=args['dec_n_layer'], nonlin=args['dec_nonlinearity'],
              weights_file=args['dec_weights'], freeze=args['dec_freeze'], latent_nonlin=args['latent_nonlinearity'])
Dis_y = Discriminator_y(model_ctx=model_ctx, batch_size=args['batch_size'], ndim_y=args['ndim_y'],
                        n_hidden=args['dis_n_hidden'], n_layers=args['dis_n_layer'],
                        nonlin=args['dis_nonlinearity'], weights_file=args['dis_y_weights'],
                        freeze=args['dis_freeze'], latent_nonlin=args['latent_nonlinearity'])

NOTE: Encoder ignoring list of hiddens because n_layer >= 0. Just using first element.
NOTE: Decoder ignoring list of hiddens because n_layer >= 0. Just using first element.


IndexError: list index out of range

In [33]:
args['enc_n_hidden']

[128]

In [34]:
args['dec_n_hidden']

[128]